# Scienti API

In [11]:
import requests
import getpass
import pandas as pd

In [35]:
asc=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vSjpIpwy89MwRwJdXNy0_Z5zCK6XrieZ2_Cf7i1SQTuk4b_hRrNzuCTBgQe3eTOew/pub?output=xlsx')

In [48]:
dasc=dict(asc[asc['ASC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

In [3]:
k=getpass.getpass('API KEY')

API KEY ········


In [9]:
institution='udea'
year=2022
p=requests.get(f"http://apis.colav.co/scienti/info/?apikey={k}&get=ids&institution={institution}&model_year={year}")

In [10]:
p=p.json()

In [14]:
pd.DataFrame(p[0].get('entities'))

,name,ids
0,patent,"[{'COD_RH': '0000204234', 'COD_PATENTE': 1}, {..."
1,product,"[{'COD_RH': '0000000011', 'COD_PRODUCTO': '1'}..."
2,network,"[{'COD_RH': '0000186473', 'COD_RED': 4}, {'COD..."
3,project,"[{'COD_RH': '0000000011', 'COD_PROYECTO': '1'}..."
4,event,"[{'COD_RH': '0000000016', 'COD_EVENTO': 1}, {'..."
5,institution_endorsement,"[{'COD_AVAL_INSTITUCION': 1382}, {'COD_AVAL_IN..."


In [17]:
p=[d for d in p[0].get('entities') if d.get('name')=='product']

In [22]:
len(p[0].get('ids'))

390012

In [25]:
d=p[0].get('ids')[100000]
d

{'COD_RH': '0000140023', 'COD_PRODUCTO': '11'}

In [32]:
w=requests.get(f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}")

In [52]:
x=w.json().get('COD_TIPO_PRODUCTO')
if isinstance(x,str) and x in dasc.keys():
    pass

In [51]:
x

'121'